# 08 · Ensembles & LiRA

Stack features from previous notebooks and compute likelihood ratio scores.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)

In [ ]:
import numpy as np

from src.attacks.ensemble import train_ensemble, predict_proba
from src.attacks.lira import compute_llr

rng = np.random.default_rng(2)
features = rng.normal(size=(64, 5))
labels = rng.integers(0, 2, size=64)

models = train_ensemble(features, labels)
probs = predict_proba(models, features)
print('Logistic mean prob:', probs['logistic'].mean())

log_probs_ft = rng.normal(size=(64, 16))
log_probs_base = rng.normal(size=(64, 16))
llr_result = compute_llr(log_probs_ft, log_probs_base)
print('LLR shape:', llr_result.llr.shape)
print('Normalized LLR sample:', llr_result.normalized_llr[:5])

**TODO:** Merge LiRA and ensemble outputs into a unified features parquet for downstream analysis.